In [45]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import sys, os
import pickle
from matplotlib import pyplot as plt
from tenpy.tools.params import asConfig
from mpomps_tenpy import *
from hubbardKitaev import Kitaev, HubbardKitaev

<font color='cyan'>
We consider a honeycomb lattice with spin-indepenten hopping terms.\
$$H=\sum_{ij}\sum_{s}-t(c_{i,s}^{\dagger} c_{j,s}^{} +h.c)$$
This lattice is implement as HoneycombLatt.
<br>The basis is $(c_{r,A,\uparrow}, c_{r,A,\downarrow}, c_{r,B,\uparrow}, c_{r,B,\downarrow})$, where $A$ and $B$ are sublattice indices.
<br>In input parameters are: 

 
    
<br> t --- hopping
<br> lx, ly --- length of unit cell along $x$, $y$ directions
<br> bcx, bcy --- boundary conditions along the $x$, $y$ directions
</font>

In [46]:
"""
We set the parameter as 
"""
lx=2; ly=3; bcy=1; bcx=0;
t=1; tx=1.; ty=1.; tz=1.;
chi=256;

In [47]:
class HoneycombLatt():

    def __init__(self, model_params=dict()):
        self.model_params = asConfig(model_params, self.__class__.__name__)
        self.t = model_params.get("t", 1.)
        self.U = model_params.get('U', 10.)
        self.lx = model_params.get("lx", 2)
        self.ly = model_params.get("ly", 2)
        self.Nlat = self.lx * self.ly * 2
        self.bcx = model_params.get('bcx', 0.)
        self.bcy = model_params.get('bcy', 1.)

        self.init_lat()

    def _xy2id(self, y, x, Y=None, X=None):
        if Y is None:
            Y = self.ly
        if X is None:
            X = self.lx
        return (y % Y) + (x % X)*Y

    def init_lat(self):
        lx, ly = self.lx, self.ly
        self.r1 = np.array([1, 0])
        self.r2 = np.array([1/2, np.sqrt(3)/2])
        self.rab = (self.r1 + self.r2) / 3.

        self.xy2id = dict()
        self.id2xy = dict()
        self.xy2nn = dict()
        self.id2nn = dict()
        self.bcxnn = set()
        self.bcynn = set()
        for _x in range(lx):
            for _y in range(ly):
                xy = (_x, _y)
                idxy = self._xy2id(_y, _x) 
                self.xy2id[xy] = idxy
                self.id2xy[idxy] = xy
                nn0 = self._xy2id(_y, _x) * 2 + 1
                nnx = self._xy2id(_y, _x) * 2 + 0
                nny = self._xy2id(_y, _x+1) * 2 + 0 
                nnz = self._xy2id(_y+1, _x) * 2 + 0
                self.id2nn[idxy] = [nnx, nny, nnz] # note the order of nn
                if _y == ly-1:
                    self.bcynn.add(  tuple(np.sort([nn0, nnz]).tolist()) )
                if _x == lx-1:
                    self.bcxnn.add(  tuple(np.sort([nn0, nny]).tolist()) )

    def calc_real_ham(self):
        tc = 1
        N, lx, ly = self.Nlat, self.lx, self.ly
        bcx, bcy = self.bcx, self.bcy
        
        ham = np.zeros((N, N), float)

        for _x in range(lx):
            for _y in range(ly):
                id0 = self._xy2id(_y,   _x)*2 + 1
                idx = self._xy2id(_y,   _x)*2 + 0
                idy = self._xy2id(_y, _x+1)*2 + 0
                idz = self._xy2id(_y+1, _x)*2 + 0

                ham[id0, idx] += -tc
                if _y == ly - 1:
                    ham[id0, idz] += -tc * bcy
                else:
                    ham[id0, idz] += -tc
                if _x == lx - 1:
                    ham[id0, idy] += -tc * bcx
                else:
                    ham[id0, idy] += -tc

        ham += ham.T.conj()
        self.ham_real = ham
        self.eng_real, self.state_real = np.linalg.eigh(ham)
        return self.eng_real, self.state_real

    def Wannier_U1(self, u, N=1):
        n, norbital = u.shape
        g1 = u.T
        position = np.diag( np.power( list(range(1,n+1)), N) )
        position12 = g1.conj() @ position @ g1.T 
        position12 = (position12 + position12.T.conj())/2.
        D, U = np.linalg.eigh(position12)
        index = np.argsort(D)
        # print(D)
        U = U[:,index]
        g3 = U.T @ g1
        index1 = np.zeros(norbital, dtype=int)
        if norbital%2 == 0:
            index1[0:(norbital):2] = np.ceil( range(0, norbital//2) ).astype(int)
            index1[1:(norbital):2] = np.ceil( range(norbital-1, norbital//2-1, -1) ).astype(int)
        else:
            index1[0:(norbital):2] = np.ceil( range(0, norbital//2+1) ).astype(int)
            index1[1:(norbital):2] = np.ceil( range(norbital-1, norbital//2, -1) ).astype(int)
        g3 = g3[index1,:]
        return g3

    def calc_wannier_state(self):
        eng, flux_vec = self.calc_real_ham()
        wannier_flux = flux_vec[:, :flux_vec.shape[1]//2]
        # wannier_flux = wannier_flux.T
        wannier_flux = self.Wannier_U1(wannier_flux)
        self.wannier_flux = wannier_flux

        N = int(self.Nlat*2); Nmode = int(self.Nlat)

        self.wannier_all = np.zeros((Nmode, N), float)
        print(self.wannier_flux.shape, self.wannier_all.shape)

        for _ in range(wannier_flux.shape[0]):
            self.wannier_all[_*2+0, 0::2] = wannier_flux[_, :]
            self.wannier_all[_*2+1, 1::2] = wannier_flux[_, :]

        return self.wannier_all, eng

In [48]:
para_honeycomb = dict(lx=lx, ly=ly, bcy=bcy, bcx=0)
model = HoneycombLatt(para_honeycomb)
uf, eng0 = model.calc_wannier_state();
print(" The wf is SU(2) symmetric ? ", (uf[1::2, 0::2] == 0).prod() * (uf[1::2, 0::2] == 0).prod() )
print(eng0)

(6, 12) (12, 24)
 The wf is SU(2) symmetric ?  1
[-2.56155281 -1.61803399 -1.61803399 -1.56155281 -0.61803399 -0.61803399
  0.61803399  0.61803399  1.56155281  1.61803399  1.61803399  2.56155281]


/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config HoneycombKitaev:
['Jx', 'Jy', 'Jz', 'bcx', 'bcy', 'lx', 'ly']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


<font color='cyan'>
We consider a honeycomb lattice with spin-orbital-coupling (SOC)
$$H=\sum_{\langle{}i,j\rangle{}\in{}a}\sum_{s,s'}-(t\sigma^0+t_a\sigma^a)_{ss'}c_{i,s}^{\dagger} c_{j,s'}^{} +h.c.,$$
where we have $x$, $y$, and $z$-type nearest-neighbor bonds. See the convention of Kitaev honeycomb model.When $t=t_a$ and inmposing an infinitely-large on-site Hubbard $U$., the above SOC Hamiltonian can lead to the Kitaev honeycomb model.\
This lattice is implement as HoneycombSOC. 
In input parameters are

    
t --- spin-indepencent hopping\
tx, ty, tz --- SOC hopping on x, y, z bonds\
lx, ly --- length of unit cell along $x$, $y$ directions\
bcx, bcy --- boundary conditions along the $x$, $y$ directions
</font>

In [49]:
class HoneycombSOC(HoneycombLatt):

    def __init__(self, model_params=dict()):
        super(HoneycombSOC, self).__init__(model_params)
        self.tx = model_params.get("tx", self.t)
        self.ty = model_params.get("ty", self.t)
        self.tz = model_params.get("tz", self.t)

    def calc_real_ham(self):
        tc = self.t
        tx, ty, tz = self.tx, self.ty, self.tz
        N, lx, ly = self.Nlat, self.lx, self.ly
        bcx, bcy = self.bcx, self.bcy
        # print("bcx: ", bcx)
        # print("bcy: ", bcy)

        ham = np.zeros((N*2, N*2), complex)

        for _x in range(lx):
            for _y in range(ly):
                id0u = self._xy2id(_y,   _x)*4 + 2
                id0d = self._xy2id(_y,   _x)*4 + 3
                idxu = self._xy2id(_y,   _x)*4 + 0
                idxd = self._xy2id(_y,   _x)*4 + 1
                idyu = self._xy2id(_y, _x+1)*4 + 0
                idyd = self._xy2id(_y, _x+1)*4 + 1
                idzu = self._xy2id(_y+1, _x)*4 + 0
                idzd = self._xy2id(_y+1, _x)*4 + 1

                # print(id0u, idxu)
                # print(id0d, idxd)
                ham[id0u, idxu] += -tc
                ham[id0d, idxd] += -tc
                ham[id0u, idxd] += -tx
                ham[id0d, idxu] += -tx
                if _y < ly - 1:
                    ham[id0u, idzu] += -tc
                    ham[id0d, idzd] += -tc
                    ham[id0u, idzu] += -tz 
                    ham[id0d, idzd] += +tz
                else:
                    ham[id0u, idzu] += -tc * bcy
                    ham[id0d, idzd] += -tc * bcy
                    ham[id0u, idzu] += -tz * bcy
                    ham[id0d, idzd] += +tz * bcy
                if _x < lx - 1:
                    ham[id0u, idyu] += -tc
                    ham[id0d, idyd] += -tc
                    ham[id0u, idyd] += - -1j*ty 
                    ham[id0d, idyu] += -  1j*ty 
                else:
                    ham[id0u, idyu] += -tc * bcx
                    ham[id0d, idyd] += -tc * bcx
                    ham[id0u, idyd] += - -1j*ty * bcx
                    ham[id0d, idyu] += -  1j*ty * bcx

        ham += ham.T.conj()
        self.ham_real = ham
        self.eng_real, self.state_real = np.linalg.eigh(ham)
        return self.eng_real, self.state_real

    def calc_wannier_state(self):
        eng, flux_vec = self.calc_real_ham()
        wannier_flux = flux_vec[:, :self.Nlat]
        # wannier_flux = wannier_flux.T
        wannier_flux = self.Wannier_U1(wannier_flux)
        self.wannier_all = wannier_flux
        return self.wannier_all, eng

<font color='cyan'>
<br> uF is the wave function for the HoneycombSOC model.
<br> We use MPO-MPS method (U1 type) to convert uF into a MPS.
<br> We construct dmrg_model and compare the MPO-MPS state (psi_soc) and DMRG state (psidmrg).
</font>

In [50]:
para_honeycomb = dict(tx=tx,ty=ty,tz=tz,t=t,lx=lx, ly=ly, bcy=1, bcx=0)
model_soc = HoneycombSOC(para_honeycomb)

ufSOC, engSOC = model_soc.calc_wannier_state();
print(engSOC[:lx*ly*2], '\n', engSOC[:lx*ly*2].sum())
print(ufSOC.shape)

uF = model_soc.wannier_all 
params_mpompsu1 = dict(cons_N="N", cons_S=None, trunc_params=dict(chi_max=128))
eng_soc = MPOMPSU1(uF, **params_mpompsu1)
eng_soc.run()
psi_soc = eng_soc.psi.copy()

/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config HoneycombSOC:
['bcx', 'bcy', 'lx', 'ly', 't', 'tx', 'ty', 'tz']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


[-4.14154902 -3.12827988 -3.12827988 -3.06118188 -2.18669827 -2.18669827
 -1.18572101 -1.06277493 -1.06277493 -0.55020461 -0.55020461 -0.26608815] 
 -22.510455446033
(12, 24)
applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 8
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 5-th mode, the fidelity is 1.0, the bond dimension is 32
applied the 6-th mode, the fidelity is 1.0, the bond dimension is 64
applied the 7-th mode, the fidelity is 1.0, the bond dimension is 128
applied the 8-th mode, the fidelity is 0.9999974719935895, the bond dimension is 128
applied the 9-th mode, the fidelity is 0.9999395411685988, the bond dimension is 128
applied the 10-th mode, the fidelity is 0.9998196673213708, the bond dimension is 128
applied the 11-th mode, the fidelity is 0.9996124798866975, the bond dimension is 128
ap

In [44]:
dmrg_params = dict(Ly=ly, Lx=lx, U=0., tx=tx, ty=ty, tz=tz, t=t)
dmrg_model = HubbardKitaev(dmrg_params)
print("vareng", dmrg_model.H_MPO.expectation_value(psi_soc))
c, E = dmrg_model.run_dmrg(chi_max=64, max_sweeps=6, init=None)
print(E)

x-bonds : [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]
y-bonds : [(1, 6), (3, 8), (5, 10)]
z-bonds : [(1, 2), (3, 4), (0, 5), (7, 8), (9, 10), (6, 11)]
vareng -22.505581113542316
init total particle number 12.0
init total Sz number 0.0


KeyboardInterrupt: 

<font color='cyan'>
We construct the exact-solution wavefunction for the Kitaev honeycomb model. It is a BCS-type parton wavefunction, whcih can be devided into $Z_2$ flux part and matter Majorana part.
<br> The $Z_2$ flux part is the shortest-range VBS (RVB) state, defined on the $x$, $y$, or $z$ bonds. 
<br> The matter Majorana part is a $p$-wave superconductiong wave function defined on the Bravais lattice of honeycomb, namly, a square lattice.
<br> The parton Hamiltonian is implemented as HoneycombKitaev

<br> Input:
<br> $J_x$, $J_y$, $J_z$ --- coupling constant of Kitaev interactions

</font>

In [38]:
def nxy2siteid(nx, ny, x, y):
    return ((x) % nx) * ny +(y) % ny
    
class HoneycombKitaev(HoneycombLatt):

    def __init__(self, model_params=dict()):
        super(HoneycombKitaev, self).__init__(model_params)
        self.Jx = model_params.get("Jx", 1.0)
        self.Jy = model_params.get("Jy", 1.0)
        self.Jz = model_params.get("Jz", 1.0)

    def Wannier_Z2(self, v, u, N=1):
        g1 = v.T
        g2 = u.T
        norbital, n = g1.shape
        position = np.power( list(range(1,n+1)), N)
        position = np.diag(position) 
        position12 = g1.conj() @ position @ g1.T + g2.conj() @ position @ g2.T
        position12 = (position12 + position12.T.conj())/2.
        D, U = np.linalg.eigh(position12)
        index = np.argsort(D)
        print(D)
        U = U[:,index]
        g3 = U.T @ g1
        g4 = U.T @ g2
        index1 = np.zeros(norbital, dtype=int)
        if norbital%2 == 0:
            index1[0:(norbital):2] = np.ceil( range(0, norbital//2) ).astype(int)
            index1[1:(norbital):2] = np.ceil( range(norbital-1, norbital//2-1, -1) ).astype(int)
        else:
            index1[0:(norbital):2] = np.ceil( range(0, norbital//2+1) ).astype(int)
            index1[1:(norbital):2] = np.ceil( range(norbital-1, norbital//2, -1) ).astype(int)
        g3 = g3[index1,:]
        g4 = g4[index1,:]
        return g3, g4

    def Kitaev4MF(self, Jxyz=None, bcy=None):
        if bcy is None:
            bcy = self.bcy
        if Jxyz is None:
            Jx, Jy, Jz = np.abs([self.Jx, self.Jy, self.Jz])
        else:
            Jx, Jy, Jz = np.abs(Jxyz)
        nx, ny = self.lx, self.ly
        pn = 1
        n = pn * (nx*ny);  # of interleaved sites

        t = np.zeros((n, n))
        d = np.zeros((n, n))
        for row in range(ny):
            for column in range(nx):
                i0  = nxy2siteid(nx, ny, column, row)
                iy  = nxy2siteid(nx, ny, column+1, row)
                iz  = nxy2siteid(nx, ny, column, row+1)
                t[i0, i0] += Jx/2
                if column < nx-1:
                    t[iy, i0] += Jy/2
                    d[iy, i0] += -Jy/2
                if row < ny-1:
                    t[iz, i0] += Jz/2
                    d[iz, i0] += -Jz/2 
                elif abs(bcy) == 1:
                    t[iz, i0] += Jz/2*bcy
                    d[iz, i0] += -Jz/2*bcy
        t += t.T.conj()
        d -= d.T
        HBdG = np.block([[t, d],[-d.conj(), -t.conj()]])
        energy, M = np.linalg.eigh(HBdG)
        print("energy", energy)
        print(energy[:n].sum()/4)
        V = M[:n,:n].T
        U = M[n:,:n].T
        return V, U

    def KitaevEX(self):
        Jxyz = [self.Jx, self.Jy, self.Jz]
        nx, ny = self.lx, self.ly
        by = self.bcy

        fV, fU = self.Kitaev4MF(Jxyz, by)
        if by == -1:
            v, u = self.Wannier_Z2(fV.T, fU.T)
            fV, fU = v, u
        if by == 1:
            v, u = self.Wannier_Z2(fV[:-1,:].T, fU[:-1,:].T)
            fV, fU = np.vstack((v, fV[-1,:])), np.vstack((u, fU[-1,:]))

        Jx, Jy, Jz = self.Jx, self.Jy, self.Jz
        a_or_c_x = -np.sign(Jx)
        a_or_c_y = -np.sign(Jy)
        a_or_c_z = -np.sign(Jz)

        pn = 4
        n = pn * (nx*ny);  # of interleaved sites

        print("*******************************generate parton wavefunction*******************************")

        # c^0 Majoranas
        V0, U0 = np.zeros((nx*ny, n), complex ), np.zeros((nx*ny, n), complex )
        count_state = 0
        for nc0 in range(fV.shape[0]):
            for iuc in range(nx*ny):
                V0[count_state, iuc*pn+0] = 1j * (fV[nc0, iuc] + fU[nc0, iuc])/2.
                V0[count_state, iuc*pn+2] = (fV[nc0, iuc] - fU[nc0, iuc])/2.
                U0[count_state, iuc*pn+0] = - 1j * (fV[nc0, iuc] + fU[nc0,iuc])/2.
                U0[count_state, iuc*pn+2] = (- fV[nc0,iuc] + fU[nc0,iuc])/2.
            count_state += 1
        print("construct the u^0 matter fermion with # of 0-degree =", count_state)
        V0 = V0[:count_state, :]
        U0 = U0[:count_state, :]

        # c^x Majoranas
        VX, UX = np.zeros((nx*ny, n), complex ), np.zeros((nx*ny, n), complex )
        count_state = 0
        for row in range(ny):
            for column in range(nx):
                ucll  = nxy2siteid(nx, ny, column, row)*pn
                ucllx = nxy2siteid(nx, ny, column, row)*pn
                VX[count_state, ucll+1] = 0.5
                VX[count_state, ucll+3] = -0.5*1j *a_or_c_x
                UX[count_state, ucll+1] = 0.5
                UX[count_state, ucll+3] = -0.5*1j *a_or_c_x
                count_state += 1;   # move to the next state 
        VX = VX[:count_state, :]
        UX = UX[:count_state, :]
        print("construct the u^x flat band with # of x-degree =", count_state)

        # c^y Majoranas
        VY, UY = np.zeros((nx*ny, n), complex ), np.zeros((nx*ny, n), complex )
        count_state = 0
        for column in range(nx-1):
            for row in range(ny):
                ucll  = nxy2siteid(nx, ny, column, row)*pn
                uclly = nxy2siteid(nx, ny, column+1, row)*pn
                VY[count_state, uclly+1] = -0.5*1j;
                VY[count_state, ucll +3] = -0.5 *a_or_c_y;
                UY[count_state, uclly+1] = 0.5*1j;
                UY[count_state, ucll +3] = 0.5 *a_or_c_y;
                count_state += 1;   # move to the next state 
        if ny % 2 == 0:
            pin_a_or_c_y = 1
            column = 0
            for row in range(ny):
                ucll = ( (column) * ny + row )*pn;
                uclly = ( (column) * ny + (row+1)%ny )*pn;
                AB = 1
                if row%2 == 0:
                    VY[count_state, ucll+AB]  = 0.5*1j ;
                    VY[count_state, uclly+AB] = -0.5*pin_a_or_c_y;
                    UY[count_state, ucll+AB]  = -0.5*1j ;
                    UY[count_state, uclly+AB] = 0.5*pin_a_or_c_y;
                    count_state += 1;   # move to the next state 
            column = nx-1
            for row in range(ny):
                ucll = ( (column) * ny + row )*pn;
                uclly = ( (column) * ny + (row+1)%ny )*pn;
                AB = 3
                if (row%2) == 0:
                    VY[count_state, ucll+AB]  = -0.5*1j ;
                    VY[count_state, uclly+AB] = -0.5*pin_a_or_c_y;
                    UY[count_state, ucll+AB]  = 0.5*1j ;
                    UY[count_state, uclly+AB] = 0.5*pin_a_or_c_y
                    count_state += 1
        else:
            pin_a_or_c_y = 1
            column = 0
            row = 0
            ucll = ( (column) * ny + row )*pn;
            uclly = ( (column) * ny + (row+1)%ny )*pn;
            AB = 1
            VY[count_state, ucll+AB]  = 0.5*1j ;
            VY[count_state, uclly+AB] = -0.5*pin_a_or_c_y;
            UY[count_state, ucll+AB]  = -0.5*1j ;
            UY[count_state, uclly+AB] = 0.5*pin_a_or_c_y;
            count_state += 1;   # move to the next state 
            
        print("construct the u^y flat band and pin down the boundary c^y Majoranas with # of y-degree =", count_state)
        VY = VY[:count_state, :]
        UY = UY[:count_state, :]

        # c^z Majoranas
        VZ, UZ = np.zeros((nx*ny, n), complex ), np.zeros((nx*ny, n), complex )
        count_state = 0
        for row in range(ny):
            for column in range(nx):
                ucll  = nxy2siteid(nx, ny, column, row)*pn
                ucllz = nxy2siteid(nx, ny, column, row+1)*pn
                if row < ny-1: 
                    VZ[count_state, ucllz+0] = -0.5;
                    VZ[count_state, ucll+2] = 0.5j *a_or_c_z;
                    UZ[count_state, ucllz+0] = -0.5;
                    UZ[count_state, ucll+2] = 0.5j *a_or_c_z;
                    count_state += 1;  
                elif abs(by) == 1:
                    VZ[count_state, ucllz+0] = -0.5;
                    VZ[count_state, ucll+2] = 0.5j*(-1+(by>-1)*2) *a_or_c_z;
                    UZ[count_state, ucllz+0] = -0.5;
                    UZ[count_state, ucll+2] = 0.5j*(-1+(by>-1)*2) *a_or_c_z;
                    count_state += 1;   # move to the next state (u^z band)
        print("construct the u^z flat band with # of z-degree =", count_state)
        # if by == -1:
        #     v, u = self.Wannier_Z2(V0.T, U0.T)
        #     V0, U0 = v, u
        # if by == 1:
        #     v, u = self.Wannier_Z2(V0[:-1,:].T, U0[:-1,:].T)
        #     V0, U0 = np.vstack((v, V0[-1,:])), np.vstack((u, U0[-1,:]))
        VU = [[VX, UX, 1], [VZ, UZ, 0], [VY, UY, 1], [V0, U0, 0]]
        return VU

    def calc_wannier_state(self):
        VU = self.KitaevEX()
        # self.eig_spin_u = np.vstack((VU[0][0],VU[1][0],VU[2][0],VU[3][0]))
        # self.eig_spin_v = np.vstack((VU[0][1],VU[1][1],VU[2][1],VU[3][1]))

        self.wannier_all_u = np.vstack((VU[0][0],VU[1][0],VU[2][0],VU[3][0]))
        self.wannier_all_v = np.vstack((VU[0][1],VU[1][1],VU[2][1],VU[3][1]))

        # self.wannier_all_u = np.vstack((VU[0][0],VU[1][0],VU[2][0]))
        # self.wannier_all_v = np.vstack((VU[0][1],VU[1][1],VU[2][1]))

        # wannier_u0, wannier_v0 = self.Wannier_Z2(VU[3][0].T, VU[3][1].T)

        # self.wannier_all_u = np.vstack((VU[0][0],VU[1][0],VU[2][0],wannier_u0))
        # self.wannier_all_v = np.vstack((VU[0][1],VU[1][1],VU[2][1],wannier_v0))

        return self.wannier_all_u, self.wannier_all_v

<font color='cyan'>
fu, fv is the particel and hole parts of wavefunction of the BCS-type state, respectively
<br> We use MPO-MPS method (Z2-type) to calculate the BCS-type parton wavefunction, followed by a Gutzwiller projection. The obtained state is gpsi_spin
</font>

In [51]:
params_spin = dict(lx=lx, ly=ly, bcy=-1, bcx=0, Jx=tx, Jy=ty, Jz=tz)
model = HoneycombKitaev(params_spin)
fu, fv = model.calc_wannier_state()
chi=256
params_mpompsz2 = dict(cons_N="Z2", cons_S=None, trunc_params=dict(chi_max=chi))
eng_spin = MPOMPSZ2(fu, fv, **params_mpompsz2)
eng_spin.run()
gpsi_spin = gutzwiller_projection(eng_spin.psi)

/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config HoneycombLatt:
['bcx', 'bcy', 'lx', 'ly']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


energy [-2.30277564e+00 -2.30277564e+00 -1.30277564e+00 -1.30277564e+00
 -1.00000000e+00 -1.62345129e-15  7.10863983e-16  1.00000000e+00
  1.30277564e+00  1.30277564e+00  2.30277564e+00  2.30277564e+00]
-2.0527756377319952
[1.47783996 2.50521544 3.47369562 4.21857272 5.12691723 5.69491241]
*******************************generate parton wavefunction*******************************
construct the u^0 matter fermion with # of 0-degree = 6
construct the u^x flat band with # of x-degree = 6
construct the u^y flat band and pin down the boundary c^y Majoranas with # of y-degree = 4
construct the u^z flat band with # of z-degree = 6
applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 5-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 6-th mode, the fi

In [52]:
model_params = dict(Ly=ly, Lx=lx,
                    tx=tx, ty=ty, tz=tz, cons_N="Z2")
model_KitaevS = Kitaev(model_params)
if model_params['cons_N'] == 'Z2':
    print( model_KitaevS.H_MPO.expectation_value(gpsi_spin) )
    # print(-17.487379551665562/4)
    model = HoneycombKitaev(params_spin)
    model.Kitaev4MF()
print(gpsi_spin.expectation_value("Sy"))
model_KitaevS.print_wilsonloop(gpsi_spin)
model_KitaevS.print_Z2flux(gpsi_spin)

-2.002084762266812
energy [-2.30277564e+00 -2.30277564e+00 -1.30277564e+00 -1.30277564e+00
 -1.00000000e+00 -1.62345129e-15  7.10863983e-16  1.00000000e+00
  1.30277564e+00  1.30277564e+00  2.30277564e+00  2.30277564e+00]
-2.0527756377319952
[ 8.51749227e-16 -2.49800181e-16 -2.70616862e-16  2.35922393e-16
 -4.61436445e-16 -4.61436445e-16  5.51642065e-16 -9.36750677e-17
 -4.64905892e-16  1.63064007e-16  1.63410951e-15 -1.19348975e-15]
wloop, ix = 0 1.0000000000000004
wloop, ix = 1 1.0000000000000004
wloop, iy = 0 -2.9568574906139958e-18
wloop, iy = 1 2.9568574906147577e-18
wloop, iy = 2 -1.0000000000000004
Z2 flux at ix = 0, iy =      0 -1.0000000000000004
Z2 flux at ix = 0, iy =      1 -2.956857490614794e-18
Z2 flux at ix = 0, iy =      2 2.956857490614103e-18


/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config HoneycombKitaev:
['Jx', 'Jy', 'Jz', 'bcx', 'bcy', 'lx', 'ly']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Z2 flux at ix = 1, iy =      0 1.0000000000000007
Z2 flux at ix = 1, iy =      1 1.0000000000000007
Z2 flux at ix = 1, iy =      2 1.0000000000000004


<font color='cyan'>
<br>We construct the a bilayer Honeycomb lattice mdoel. The first layer is the HoneycombSOC. The second layer is a ancilla-fermion layer. The Hamiltonian is
$$
\begin{aligned}
    H=&\sum_{\langle{}i,j\rangle{}\in{}a}\sum_{s,s'}\left[-(t\sigma^0+t_a\sigma^a)_{ss'}c_{i,s}^{\dagger} c_{j,s'}^{} +h.c.\right]+ {\rm chemical~potential}+\\
    &\sum_{\langle{}i,j\rangle{}\in{}a}\sum_{s}-t'(c_{i,s}^{\dagger} c_{j,s}^{} +h.c.) + {\rm chemical~potential}+\\ &\Phi\sum_{i}\left(c^\dagger_{i,s}f^{}{i,s}+f^\dagger_{i,s}c^{}_{i,s}\right),
\end{aligned}
$$

<br>The basis is $(c_{r,A,\uparrow}, c_{r,A,\downarrow}, f_{r,A,\uparrow}, f_{r,A,\downarrow}, c_{r,B,\uparrow}, c_{r,B,\downarrow},f_{r,B,\uparrow}, f_{r,B,\downarrow})$, where $A$ and $B$ are sublattice indices.

<br> Input:
<br> $\Phi$ --- coupling between $c$-fermion and $f$-fermion.
</font>

In [41]:
class FluxHoneycomb(HoneycombLatt):

    def __init__(self, model_params=dict()):
        super(FluxHoneycomb, self).__init__(model_params)
        self.Pcf = model_params.get("P", np.inf)
        self.tf = model_params.get("tf", 0.)
        self.tc = self.t
        self.tx = model_params.get("tx", self.t)
        self.ty = model_params.get("ty", self.t)
        self.tz = model_params.get("tz", self.t)
        self.uf = model_params.get('uf', 0.)
        self.uc = model_params.get('uc', 0.)

    def calc_real_ham(self):
        tc, tf, uc, uf, Pcf = self.tc, self.tf, self.uc, self.uf, self.Pcf
        tx, ty, tz = self.tx, self.ty, self.tz
        N, lx, ly = self.Nlat, self.lx, self.ly
        bcx, bcy = self.bcx, self.bcy

        ham = np.zeros((N*4, N*4), complex)

        for _x in range(lx):
            for _y in range(ly):
                id0 = 8*self._xy2id(_y,  _x)
                for indx in [0, 1, 4, 5]:
                    ham[id0+indx, id0+indx] += uc/2
                for indx in [2, 3, 6, 7]:
                    ham[id0+indx, id0+indx] += uf/2
                for indx in [0, 1, 4, 5]:
                    ham[id0+indx, id0+indx+2] += Pcf

                id0A = 8*self._xy2id(_y,  _x) + 0
                id0B = 8*self._xy2id(_y,  _x) + 4
                idzA = 8*self._xy2id(_y+1,  _x) + 0
                idyA = 8*self._xy2id(_y,  _x+1) + 0

                ham[id0B+0, id0A+0] += -tc
                ham[id0B+1, id0A+1] += -tc
                ham[id0B+0, id0A+1] += -tx
                ham[id0B+1, id0A+0] += -tx
                ham[id0B+2, id0A+2] += +tf
                ham[id0B+3, id0A+3] += +tf

                if _y == ly - 1:
                    ham[id0B+0, idzA+0] += -(tc+tz)*bcy
                    ham[id0B+1, idzA+1] += -(tc-tz)*bcy
                    ham[id0B+2, idzA+2] += +tf*bcy
                    ham[id0B+3, idzA+3] += +tf*bcy
                else:
                    ham[id0B+0, idzA+0] += -(tc+tz)
                    ham[id0B+1, idzA+1] += -(tc-tz)
                    ham[id0B+2, idzA+2] += +tf
                    ham[id0B+3, idzA+3] += +tf
                if _x == lx - 1:
                    ham[id0B+0, idyA+0] += -tc
                    ham[id0B+1, idyA+1] += -tc
                    ham[id0B+0, idyA+1] += - -1j*ty
                    ham[id0B+1, idyA+0] += - +1j*ty
                    ham[id0B+2, idyA+2] += +tf
                    ham[id0B+3, idyA+3] += +tf
                else:
                    ham[id0B+0, idyA+0] += -tc
                    ham[id0B+1, idyA+1] += -tc
                    ham[id0B+0, idyA+1] += - -1j*ty
                    ham[id0B+1, idyA+0] += - +1j*ty
                    ham[id0B+2, idyA+2] += +tf
                    ham[id0B+3, idyA+3] += +tf

        ham += ham.T.conj()
        self.ham_real = ham
        self.eng_real, self.state_real = np.linalg.eigh(ham)
        return self.eng_real, self.state_real

    def calc_wannier_state(self):
        eng, flux_vec = self.calc_real_ham()
        wannier_flux = flux_vec[:, :flux_vec.shape[1]//2]
        # wannier_flux = wannier_flux.T
        wannier_flux = self.Wannier_U1(wannier_flux)
        self.wannier_all = wannier_flux
        return self.wannier_all, eng

    def calc_u_real(self, Pcf=None):
        if Pcf is None:
            Pcf = self.Pcf
        if Pcf == np.inf:
            N, lx, ly = self.Nlat, self.lx, self.ly
            self.state_real = np.zeros((N*4, N*2), float)
            rs2 = 1/np.sqrt(2)
            for _ in range(N):
                id0 = _ * 4
                self.state_real[id0+0, _*2+0] = +rs2
                self.state_real[id0+2, _*2+0] = -rs2
                self.state_real[id0+1, _*2+1] = +rs2
                self.state_real[id0+3, _*2+1] = -rs2
                # self.state_real[id0+0, _*2+N*2+0] = +rs2
                # self.state_real[id0+2, _*2+N*2+0] = +rs2
                # self.state_real[id0+1, _*2+N*2+1] = +rs2
                # self.state_real[id0+3, _*2+N*2+1] = +rs2
            self.state_real = self.state_real.T
            return self.state_real
        elif 1e-12 < abs(self.Pcf) < self.tc:
            print("Warning! ")
        elif abs(self.Pcf) < 1e-12:
            N, lx, ly = self.Nlat, self.lx, self.ly
            self.state_real = np.zeros((N*4, N*2), complex)
            rs2 = 1/np.sqrt(2)
            paraSOC = dict(t=self.t, tx=self.tx, ty=self.ty, tz=self.tz,
                           lx=self.lx, ly=self.ly, bcx=self.bcx, bcy=self.bcy)
            m = HoneycombSOC(paraSOC)
            m.calc_wannier_state()
            uf = m.wannier_all
            self.state_real = np.zeros((N*4, N*2), complex)
            for _ in range(N//2):
                self.state_real[_*8+2, _*2+0] = +rs2
                self.state_real[_*8+6, _*2+0] = -rs2
                self.state_real[_*8+3, _*2+1] = +rs2
                self.state_real[_*8+7, _*2+1] = -rs2
            for _ in range(uf.shape[0]):
                self.state_real[0::4, N+_] = uf[_, 0::2]
                self.state_real[1::4, N+_] = uf[_, 1::2]
            self.state_real = self.state_real.T
            return self.state_real
        else:
            # self.calc_real_ham()
            self.calc_wannier_state()
            return self.wannier_all

In [42]:
params_flux = dict(lx=lx, ly=ly, bcy=bcy, bcx=0, tx=tx, ty=ty, tz=tz, tc=t, tf=0, uc=0, uf=0, P=np.inf)
model_flux = FluxHoneycomb(params_flux)
fu = model_flux.calc_u_real()
params_mpompsu1 = dict(cons_N="N", cons_S=None, trunc_params=dict(chi_max=chi))
eng_flux = MPOMPSU1(fu, **params_mpompsu1)
eng_flux.run()
psi_flux = eng_flux.psi
sp_psi = singlet_projection(psi_flux, gpsi_spin)

/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config FluxHoneycomb:
['P', 'bcx', 'bcy', 'lx', 'ly', 'tc', 'tf', 'tx', 'ty', 'tz', 'uc', 'uf']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 5-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 6-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 7-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 8-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 9-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 10-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 11-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 12-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 13-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 14-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 15-th mode, the fidelity is 1.0

In [43]:
model_params = dict(Ly=ly, Lx=lx,
                    tx=tx, ty=ty, tz=tz, cons_N="N")
model_KitaevSN = Kitaev(model_params)
print("results for MPO-MPS")
print( model_KitaevSN.H_MPO.expectation_value(sp_psi) )
model_KitaevSN.print_wilsonloop(sp_psi)
model_KitaevSN.print_Z2flux(sp_psi)
psidmrgN, E = model_KitaevSN.run_dmrg(chi_max=128, max_sweeps=8, init=None)
print(E)
# print("results for DMRG")
# model_KitaevSN.print_wilsonloop(psidmrgN)
# model_KitaevSN.print_Z2flux(psidmrgN)
# print( psidmrgN.overlap(sp_psi) )

results for MPO-MPS
-2.00208476226681
wloop, ix = 0 0.015625
wloop, ix = 1 0.015624999999999998
wloop, iy = 0 -7.518778600865035e-19
wloop, iy = 1 7.518778600865416e-19
wloop, iy = 2 -0.06250000000000003
Z2 flux at ix = 0, iy =      0 -0.015624999999999997
Z2 flux at ix = 0, iy =      1 -1.879694650216362e-19
Z2 flux at ix = 0, iy =      2 1.8796946502162786e-19
Z2 flux at ix = 1, iy =      0 0.015625
Z2 flux at ix = 1, iy =      1 0.015625
Z2 flux at ix = 1, iy =      2 0.015625
init total Sz number 5.551115123125783e-17
-1.5872744757321184
-1.5872744757321184


In [21]:
from PartonHoneycomb import FluxHoneycomb
params_flux = dict(lx=lx, ly=ly, bcy=bcy, bcx=0, tx=tx, ty=ty, tz=tz, tc=t, tf=0, uc=0, uf=0, P=0)
model_flux = FluxHoneycomb(params_flux)
fu = model_flux.calc_u_real()
params_mpompsu1 = dict(cons_N="N", cons_S=None, trunc_params=dict(chi_max=chi))
eng_flux0 = MPOMPSU1(fu, **params_mpompsu1)
eng_flux0.run()
psi_flux0 = eng_flux0.psi
sp_psi2 = singlet_projection(psi_flux0, gpsi_spin)

/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config FluxHoneycomb:
['P', 'bcx', 'bcy', 'lx', 'ly', 'tc', 'tf', 'tx', 'ty', 'tz', 'uc', 'uf']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config HoneycombSOC:
['bcx', 'bcy', 'lx', 'ly', 't', 'tx', 'ty', 'tz']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 5-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 6-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 7-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 8-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 9-th mode, the fidelity is 1.0, the bond dimension is 8
applied the 10-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 11-th mode, the fidelity is 1.0, the bond dimension is 32
applied the 12-th mode, the fidelity is 1.0, the bond dimension is 64
applied the 13-th mode, the fidelity is 1.0, the bond dimension is 128
applied the 14-th mode, the fidelity is 1.0, the bond dimension is 256
applied the 15-th mode, the fidelity

In [22]:
from hubbardKitaev import HubbardKitaev
dmrg_params = dict(Ly=ly, Lx=lx, U=0., tx=tx, ty=ty, tz=tz, t=t)
dmrg_model = HubbardKitaev(dmrg_params)
print("vareng", dmrg_model.H_MPO.expectation_value(sp_psi2))

x-bonds : [(0, 1), (2, 3), (4, 5), (6, 7)]
y-bonds : [(1, 4), (3, 6)]
z-bonds : [(1, 2), (0, 3), (5, 6), (4, 7)]
vareng -14.947066831227364
